In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import json
from pandas import json_normalize
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium import Choropleth
import matplotlib
from datetime import datetime 
# GetCompasData로 주피터 노트북 경로에 파일 저장
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
matplotlib.rcParams['font.family']="Malgun Gothic"
import pydeck as pdk
import mapboxgl
mapboxgl.__version__
from mapboxgl.viz import *
import os
from mapboxgl.utils import create_color_stops
from mapboxgl.utils import create_numeric_stops
import jenkspy
from sklearn.preprocessing import MinMaxScaler

In [2]:
#원래 데이터
suwon_112 = pd.read_csv('1.수원시_112신고이력(격자_관할경계매핑).csv')
others_112 = pd.read_csv('3.3개시도_112신고이력(관할경계매핑).csv')
others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')
others_police_area = gpd.read_file('11.4개시도_경찰서_관할경계.geojson')
others_police_box_area = gpd.read_file('12.4개시도_치안안전센터_관할경계.geojson')
others_security_lamp = pd.read_csv('22.4개시도_보안등현황.csv')
others_security_bell = gpd.read_file('23.4개시도_안전비상벨현황.geojson')
others_cctv = pd.read_csv('24.4개시도_CCTV설치현황.csv')
suwon_building = pd.read_csv('26.수원시_건축물대장(표제부).csv')
others_park = pd.read_csv('27.4개시도_공원현황.csv')
others_police = pd.read_csv('39.4개시도_경찰관서현황.csv')
others_playground = pd.read_csv('40.4개시도_어린이놀이시설현황.csv')
others_land_use_plan = gpd.read_file('41.4개시도_토지이용계획도.geojson')
others_convenience = gpd.read_file('42.4개시도_편의점현황.geojson')



#구,동별로 되어 있는 데이터
suwon_homeless = pd.read_csv('13.수원시_노숙인자활시설현황.csv')
suwon_child_welfare = pd.read_csv('14.수원시_아동복지시설현황.csv')
suwon_child_center = pd.read_csv('15.수원시_지역아동센터시설현황.csv')
suwon_youth_study = pd.read_csv('16.수원시_청소년공부방현황.csv')
suwon_library = pd.read_csv('17.수원시_도서관현황.csv')
suwon_free_cafeteria =pd.read_csv('18.수원시_무료급식소현황.csv')
suwon_child_free_cafeteria = pd.read_csv('19.수원시_아동복지급식소현황.csv')
suwon_female_parcel = pd.read_csv('20.수원시_여성안심택배함현황.csv')
suwon_street_lamp = pd.read_csv('21.수원시_가로등현황.csv')


# 병원, 약국 데이터
병원_4개시 = pd.read_csv('4개시도_병원.csv')
약국_4개시 = pd.read_csv('4개시도_약국.csv')

### 치안센터 기준의 데이터프레임 생성

In [3]:
others_police_box_area.columns=['관할치안센터','관할경찰서','geometry']
others_police_box_area=others_police_box_area.drop_duplicates(['관할치안센터'])
others_police_box_area.reset_index(drop=True,inplace=True)

In [4]:
police_box_4sido = others_police_box_area.copy()
police_box_4sido = police_box_4sido[['관할치안센터','관할경찰서','geometry']]
police_box_4sido.sort_values(by=["관할치안센터"], inplace=True)
police_box_4sido.reset_index(inplace = True,drop=True)

### mapping & count 함수 생성

In [5]:
def point_police_box(data_df,oters_police_box_area,col):
    df = data_df.copy()

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    police_box_result = gpd.sjoin(others_police_box_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_box_result_groupby = police_box_result.groupby("관할치안센터").agg({col:"count"}).reset_index().rename(columns={col:"cnt"})
    
    return police_box_result,police_box_result_groupby

# 카운트용 함수

def point_police_box_count(data_df,others_police_box,col):
    df = data_df.copy()

    epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
    df['geometry'] = df.apply(lambda x : Point([x['lon'], x['lat']]), axis=1)
    df = gpd.GeoDataFrame(df, geometry='geometry', crs = epsg4326)
    
    police_box_result = gpd.sjoin(others_police_box_area, df, how='inner', op="intersects").drop(["index_right"],axis=1)
    
    police_box_result_groupby = police_box_result.groupby("관할치안센터").agg({col:"sum"}).reset_index().rename(columns={col:"cnt"})
    
    return police_box_result,police_box_result_groupby    
def police_box_4sido_count(df,name):
    for i in range(len(df)):
        for j in range(len(police_box_4sido)):
            if df.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터']:
                    police_box_4sido.loc[j, name]=df.loc[i, 'cnt']

#### 지역명 매핑

In [6]:
area_list = []
for i in range(len(police_box_4sido)):
    area_list.append(police_box_4sido['관할경찰서'][i][0:2])
for i in range(len(area_list)):
    if (area_list[i]=='마산') | (area_list[i] == '진해'):
        area_list[i] = '창원'
police_box_4sido['지역'] = area_list


### 27.4개시도_공원현황 mapping

In [7]:
others_park_result_1, others_park_groupby=point_police_box(others_park,others_police_box_area,'park_gbn')
others_park_result=others_park_result_1.groupby(['관할치안센터', 'park_gbn']).count()
others_park_result.reset_index(inplace=True)
others_park_result=others_park_result[['관할치안센터','park_gbn','park_nm']]
police_box_4sido_count(others_park_groupby,'공원')

### 40.4개시도_어린이놀이시설현황/40.4개시도_어린이놀이시설현황_구별종류별

In [8]:
others_playground_result, others_playground_groupby=point_police_box(others_playground,others_police_box_area,'fac_loc')
police_box_4sido_count(others_playground_groupby,'놀이시설')

### 42.4개시도_편의점현황/42.4개시도_편의점현황_업체별수

In [9]:
others_convenience_result, others_convenience_groupby=point_police_box(others_convenience,others_police_box_area,'KSIC_NM')
others_convenience_result=others_convenience_result.groupby(['관할치안센터', 'KSIC_NM']).count()
others_convenience_result.reset_index(inplace=True)
others_convenience_result=others_convenience_result[['관할치안센터','KSIC_NM','BUSINESS_NM']]
police_box_4sido_count(others_convenience_groupby,'편의점')

In [10]:
others_convenience.rename(columns={'LON':'lon', 'LAT':'lat'}, inplace=True)

### 22.4개시도_보안등현황 mapping

In [11]:
others_security_lamp_result, others_security_lamp_groupby=point_police_box_count(others_security_lamp,others_police_box_area,'securitylight_cnt')
police_box_4sido_count(others_security_lamp_groupby,'보안등')

### 23.4개시도_안전비상벨현황 mapping

In [12]:
others_security_bell_result = gpd.sjoin(others_police_box_area, others_security_bell, how='inner', op="intersects").drop(["index_right"],axis=1)
others_security_bell_groupby = others_security_bell_result.groupby("관할치안센터").agg({'bell_us':"count"}).reset_index().rename(columns={'bell_us':"cnt"})
others_security_bell_result=others_security_bell_result.groupby(['관할치안센터', 'bell_us'])['관할경찰서'].count()
others_security_bell_result=others_security_bell_result.to_frame().reset_index().rename(columns={'관할경찰서':"cnt"})
police_box_4sido_count(others_security_bell_groupby,'비상벨')

### 24.4개시도_CCTV설치현황 mapping

In [13]:
others_cctv_result,others_cctv_groupby = point_police_box_count(others_cctv,others_police_box_area,'cctv_cnt')
others_cctv_result = others_cctv_result.groupby(['관할치안센터','purpose'])['cctv_cnt'].sum()
others_cctv_result=others_cctv_result.to_frame().reset_index()
police_box_4sido_count(others_cctv_groupby,'cctv')

### 112 신고이력 관련 데이터 가공
- 20개 이상의 사건종별 데이터를 6개의 중분류로 매핑해주기
- 교통 / 질서유지 / 중요범죄 / 기타범죄 / 기타_경찰업무 / 타기관_기타

### 112 데이터 전처리 함수

In [14]:
def 사건종별_분류(x):
    교통 = ["교통사고","교통불편","교통위반"]
    질서유지 = ["기타경범","주취자","행패소란","보호조치","시비","위험방지","청소년비행","무전취식승차"]
    중요범죄 = ["절도","가정폭력","폭력"]
    기타범죄 = ["기타형사범","도박"]
    기타_경찰업무 = ["상담문의","분실습득","비상벨","실종(실종아동 등)"]
    타기관_기타 = ["기타_타기관","소음","서비스요청"]
    
    if x in 교통:
        return "교통"
    if x in 질서유지:
        return "질서유지"
    if x in 중요범죄:
        return "중요범죄"
    if x in 기타범죄:
        return "기타범죄"
    if x in 기타_경찰업무:
        return "기타_경찰업무"
    if x in 타기관_기타:
        return "타기관_기타"

In [15]:
def time_cut(x):
    if 6 > x >= 2:
        return '새벽'
    elif 11 > x >= 6:
        return '오전'
    elif 15 > x >= 11:
        return '점심'
    elif 18 > x >= 15:
        return '오후'
    elif 22 > x >= 18:
        return '저녁'
    else:
        return '심야'

In [16]:
others_112.columns = ["사건종별","접수완료일자","접수완료시각","도착일시","관할치안센터","관할경찰서"]
suwon_112.columns = ["사건종별","접수완료일자","접수완료시각","도착일시","gid","관할치안센터","관할경찰서"]

In [17]:
def 신고이력데이터_처리(data):
    
    others_112 = data.copy()
    others_112["접수완료시각"] = others_112["접수완료시각"].str.replace("'","")
    others_112["접수완료일자"] = others_112["접수완료일자"].astype(str)
    #others_112["도착일시"] = others_112["도착일시"].astype(int).astype(str)
    # 도착일시가 NaN인 경우 존재 -> 도착일시가 NaN이 아닌 경우에 대해서만 출동 소요시각 구하기

    others_112["접수완료일시"] = others_112["접수완료일자"] + others_112["접수완료시각"]
    others_112["접수완료일시"] = pd.to_datetime(others_112["접수완료일시"],format="%Y%m%d%H%M%S")

    others_112["년"] = others_112["접수완료일시"].apply(lambda x : x.year)
    others_112["월"] = others_112["접수완료일시"].apply(lambda x : x.month)
    others_112["일"] = others_112["접수완료일시"].apply(lambda x : x.day)
    others_112["요일"] = others_112["접수완료일시"].apply(lambda x : x.weekday())
    others_112["시"] = others_112["접수완료일시"].apply(lambda x : x.hour)
    others_112["분"] = others_112["접수완료일시"].apply(lambda x : x.minute)
    others_112["초"] = others_112["접수완료일시"].apply(lambda x : x.second)

    # 도착일시가 정확히 기록된 경우와 그렇지 않은 경우로 나누기

    def get_도착일시(x):
        if len(str(x)) < 14:
            return np.NaN
        else:
            return str(int(float(x)))

    others_112["도착일시"] = others_112["도착일시"].apply(lambda x : get_도착일시(x))
    others_112["도착일시"] = pd.to_datetime(others_112["도착일시"],format="%Y%m%d%H%M%S")

    # 출동하는데 걸린 소요 시간
    others_112["소요시간"] = others_112["도착일시"] - others_112["접수완료일시"]

    # 사건종별 중분류
    others_112["사건종별_중분류"] = others_112["사건종별"].apply(lambda x : 사건종별_분류(x))
    
    # 시간대별 분류
    others_112["시간대"] = others_112["시"].apply(lambda x : time_cut(x))
    
    return others_112

In [18]:
suwon_112_get = 신고이력데이터_처리(suwon_112)
others_112_get = 신고이력데이터_처리(others_112)

suwon_112_groupby = suwon_112_get.groupby("관할치안센터").agg({'gid':"count"}).reset_index().rename(columns={'gid':"cnt"})
suwon_112_get_groupby = suwon_112_get.groupby(['관할치안센터','사건종별_중분류'])['gid'].count().reset_index().rename(columns={'gid':"cnt"})

others_112_groupby = others_112_get.groupby("관할치안센터").agg({'사건종별':"count"}).reset_index().rename(columns={'사건종별':"cnt"})
others_112_get_groupby = others_112_get.groupby(['관할치안센터','사건종별_중분류'])['사건종별'].count().reset_index().rename(columns={'사건종별':"cnt"})

### 112 신고 데이터 치안센터 영역과 매핑하는 함수

In [19]:
def police_box_4sido_112_mapping(df1,df2):
    police_box_4sido_count(df1,'전체112')
    for i in range(len(df2)):
        for j in range(len(police_box_4sido)):
            if df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '기타_경찰업무':
                police_box_4sido.loc[j, '112_기타_경찰업무']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '중요범죄':
                police_box_4sido.loc[j, '112_중요범죄']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '교통':
                police_box_4sido.loc[j, '112_교통']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '타기관_기타':
                police_box_4sido.loc[j, '112_타기관_기타']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '질서유지':
                police_box_4sido.loc[j, '112_질서유지']=df2.loc[i, 'cnt']
            elif df2.loc[i, '관할치안센터'] == police_box_4sido.loc[j, '관할치안센터'] and df2.loc[i, '사건종별_중분류'] == '기타범죄':
                police_box_4sido.loc[j, '112_기타범죄']=df2.loc[i, 'cnt']


In [20]:
police_box_4sido['전체112']=0
police_box_4sido['112_기타_경찰업무']=0
police_box_4sido['112_중요범죄']=0
police_box_4sido['112_교통']=0
police_box_4sido['112_타기관_기타']=0
police_box_4sido['112_질서유지']=0
police_box_4sido['112_기타범죄']=0

In [21]:
police_box_4sido_112_mapping(suwon_112_groupby,suwon_112_get_groupby)

In [22]:
police_box_4sido_112_mapping(others_112_groupby,others_112_get_groupby)

In [23]:
police_box_4sido['112_중요범죄_비율'] = round((police_box_4sido['전체112']/police_box_4sido['112_중요범죄'])*100)

### 8. 4개시도_유흥업소현황 mapping

In [24]:
others_alcohol = gpd.read_file('8.4개시도_유흥업소현황.geojson')[['BUSINESS_NM','COM_LC_NM','COM_MC_NM',
                'COM_SC_NM','KSIC_NM','SGG_NM',
       'ADB_EMD_NM','EMD_NM','ROAD_NM','geometry']]
others_alcohol.columns = ["상호명","상권업종대분류명","상권업종중분류명","상권업종소분류명","표준산업분류명",
                         "시군구명","행정동명","법정동명","도로명","geometry"]

In [25]:
others_alcohol_result = gpd.sjoin(others_police_box_area, others_alcohol, how='inner', op="intersects").drop(["index_right"],axis=1)
others_alcohol_result=others_alcohol_result.groupby(['관할치안센터'])['관할경찰서'].count().to_frame().reset_index().rename(columns={'관할경찰서':"cnt"})
police_box_4sido_count(others_alcohol_result,'유흥업소')

### police_box_4sido 처리

In [26]:
police_box_4sido.fillna(0,inplace=True)
ar = ['공원', '놀이시설', '편의점', '보안등', '비상벨', 'cctv','전체112', '112_기타_경찰업무', '112_중요범죄', '112_교통', '112_타기관_기타', '112_질서유지','112_기타범죄', '유흥업소']
for i in ar:
    police_box_4sido[i] = police_box_4sido[i].astype(int)

In [27]:
#경찰서 기준 데이터프레임 생성
police_4sido = police_box_4sido.groupby('관할경찰서').apply(lambda x: x.sum())
police_4sido=police_4sido.drop(['관할치안센터','관할경찰서','지역','112_중요범죄_비율'],axis=1).reset_index()

In [28]:
#경찰서 기준 데이터프레임에 지역 매핑
police_4sido['지역'] =police_4sido['관할경찰서'].str[:2]
police_4sido['지역']=police_4sido['지역'].replace('마산','창원')
police_4sido['지역']=police_4sido['지역'].replace('진해','창원')

In [29]:
# #4개시_치안센터_final to_file
# police_box_4sido.to_file("4개시_치안센터_final.geojson",driver="GeoJSON")

In [30]:
#4개시_경찰서_final geojson변환
others_police_area=others_police_area.rename(columns={'NAME':'관할경찰서'}).drop('PNAME',axis=1)
police_4sido = pd.merge(others_police_area,police_4sido, on ='관할경찰서' )

### 보안등 null값 처리 
- 평균값: 5522
- 하위 25%: 3927
- 2000이하인 값들은 이상치로 분류하여 해당지역의 평균값으로 대체

In [31]:
#보안등_이상치 처리
li = list(police_4sido[police_4sido['보안등'] < 2000].index)
police_4sido['보안등_이상치']=0
police_4sido.loc[li,'보안등_이상치'] = 1

In [32]:
police_4sido.loc[(police_4sido['보안등'] < 2000)&(police_4sido['지역']=='서울'), '보안등'] = round(police_4sido[(police_4sido['보안등']>=2000)&(police_4sido['지역']=='서울')]['보안등'].mean())

### cctv null값 처리 
- 평균값: 2398
- 하위 25%: 1376
- 500이하인 값들은 이상치로 분류하여 해당지역의 평균값으로 대체

In [33]:
#cctv_이상치 처리
li = list(police_4sido[police_4sido['cctv'] < 500].index)
police_4sido['cctv_이상치']=0
police_4sido.loc[li,'cctv_이상치'] = 1

In [34]:
police_4sido.loc[(police_4sido['cctv'] <500)&(police_4sido['지역']=='서울'), 'cctv'] = round(police_4sido[(police_4sido['cctv']>=500)&(police_4sido['지역']=='서울')]['cctv'].mean())

### 비상벨 null값 처리 
- 평균값: 455
- 하위 25%: 28
- 10이하인 값들은 이상치로 분류하여 해당지역의 평균값으로 대체

In [35]:
#비상벨_이상치 처리
li = list(police_4sido[police_4sido['비상벨'] < 10].index)
police_4sido['비상벨_이상치']=0
police_4sido.loc[li,'비상벨_이상치'] = 1

In [36]:
police_4sido.loc[(police_4sido['비상벨'] <10)&(police_4sido['지역']=='서울'), '비상벨'] = round(police_4sido[(police_4sido['비상벨']>=10)&(police_4sido['지역']=='서울')]['비상벨'].mean())

### 경찰서당 인구 매핑

In [37]:
pop_result = []
file_list = ["강동구인구/nlsp_021001001.shp","강북구인구/nlsp_021001001.shp","중랑구인구/nlsp_021001001.shp"
            ,"강서구인구/nlsp_021001001.shp","관악구인구/nlsp_021001001.shp","광진구인구/nlsp_021001001.shp"
            ,"구로구인구/nlsp_021001001.shp","금천구인구/nlsp_021001001.shp","노원구인구/nlsp_021001001.shp"
            ,"도봉구인구/nlsp_021001001.shp","동대문구인구/nlsp_021001001.shp","동작구인구/nlsp_021001001.shp"
            ,"마포구인구/nlsp_021001001.shp","서대문구인구/nlsp_021001001.shp","서초구인구/nlsp_021001001.shp"
            ,"성동구인구/nlsp_021001001.shp","성북구인구/nlsp_021001001.shp","송파구인구/nlsp_021001001.shp"
            ,"양천구인구/nlsp_021001001.shp","영등포구인구/nlsp_021001001.shp","용산구인구/nlsp_021001001.shp"
            ,"은평구인구/nlsp_021001001.shp","종로구인구/nlsp_021001001.shp","중구인구/nlsp_021001001.shp"
            ,"수원시인구/nlsp_021001001.shp","진주시인구/nlsp_021001001.shp","창원시인구/nlsp_021001001.shp"]


In [38]:
강남_인구 = gpd.read_file("강남구인구/nlsp_021001001.shp",encoding="utf-8").to_crs(epsg="4326")[["gid","val","geometry"]]
강남_인구_centroid = gpd.GeoDataFrame(강남_인구,geometry = 강남_인구.geometry.centroid)
강남_gid_with_police = gpd.sjoin(강남_인구_centroid, police_4sido[["관할경찰서","geometry"]],
                                       how="inner", op="intersects").drop("index_right",axis=1)
pop_result=강남_gid_with_police.groupby("관할경찰서").agg("sum")["val"].reset_index()

for i in file_list:
    a = gpd.read_file(i,encoding="utf-8").to_crs(epsg="4326")[["gid","val","geometry"]]
    a = gpd.GeoDataFrame(a,geometry = a.geometry.centroid)
    a = gpd.sjoin(a, police_4sido[["관할경찰서","geometry"]],
                                           how="inner", op="intersects").drop("index_right",axis=1)
    a=a.groupby("관할경찰서").agg("sum")["val"].reset_index()
    pop_result = pd.concat([pop_result,a])
pop_result = pop_result.groupby('관할경찰서').agg('sum')
pop_result.reset_index(inplace=True)

In [39]:
for i in range(len(pop_result)):
    for j in range(len(police_4sido)):
        if pop_result.loc[i,'관할경찰서'] == police_4sido.loc[j,'관할경찰서']:
            police_4sido.loc[j,'인구'] = pop_result.loc[i,'val']

### 단위면적당,단위인구당 112신고 데이터, 시설물 

In [40]:
police_4sido = police_4sido.to_crs(epsg="5186")
police_4sido["area"] = police_4sido["geometry"].apply(lambda x :round(x.area,3))
# Polygon 영역 넓이 구하기

police_4sido = police_4sido.to_crs(epsg="4326")
# 좌표계 반환

### 분석 관련 feature 제외 drop

In [41]:
police_4sido=police_4sido[['관할경찰서','지역', '공원', '놀이시설', '편의점', '보안등', '비상벨', 'cctv', '전체112',
        '112_교통','112_질서유지','유흥업소','보안등_이상치', 'cctv_이상치','비상벨_이상치', 'area','인구', 'geometry']]

In [42]:
# 112데이터/면적
li = ['112_교통','112_질서유지','공원','놀이시설','편의점','보안등','비상벨','cctv','유흥업소']
for i in li:
    police_4sido['{}/면적'.format(i)] = police_4sido[i]/police_4sido['area']
#Feature/인구
for i in li:
    police_4sido['{}/인구'.format(i)] = police_4sido[i]/police_4sido['인구']

### 각 Feature들 스케일링
- 단위면적당, 단위인구당 112 신고 데이터와 시설물 데이터를 스케일링

In [43]:
a = police_4sido[['관할경찰서','지역','area','geometry']]
police_4sido = police_4sido[['112_교통/면적', '112_질서유지/면적', '공원/면적', '놀이시설/면적',
       '편의점/면적', '보안등/면적', '비상벨/면적', 'cctv/면적', '유흥업소/면적','112_교통/인구',
       '112_질서유지/인구', '공원/인구', '놀이시설/인구', '편의점/인구', '보안등/인구', '비상벨/인구',
       'cctv/인구', '유흥업소/인구']]

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(police_4sido)
b = scaler.transform(police_4sido)
police_4sido = pd.DataFrame(b, columns=police_4sido.columns, index=list(police_4sido.index.values))
police_4sido = pd.concat([police_4sido,a],axis=1)

 ### 각 경찰서에 해당하는 단위면적당,단위인구당 112 신고 데이터와 시설물 데이터를 계산

In [44]:
for i in ['112_교통','112_질서유지','공원','놀이시설','편의점','보안등','비상벨','cctv','유흥업소']:
    police_4sido[i] = police_4sido['{}/면적'.format(i)]+police_4sido['{}/인구'.format(i)]
police_4sido = police_4sido[['관할경찰서','지역', '112_교통', '112_질서유지', '공원', '놀이시설', '편의점',
       '보안등', '비상벨', 'cctv', '유흥업소','area', 'geometry']]
police_4sido

,관할경찰서,지역,112_교통,112_질서유지,공원,놀이시설,편의점,보안등,비상벨,cctv,유흥업소,area,geometry
0,진주경찰서,진주,0.167511,0.050306,0.489059,0.859364,0.088949,0.040168,0.255390,0.167373,0.756132,7.127162e+08,"MULTIPOLYGON (((128.26697 35.12927, 128.26697 ..."
1,창원서부경찰서,창원,0.166710,0.072394,1.045381,0.786503,0.096007,0.090240,0.000089,0.071355,1.069676,1.964036e+08,"MULTIPOLYGON (((128.63363 35.22152, 128.63357 ..."
2,창원중부경찰서,창원,0.209198,0.116303,0.925835,0.546646,0.117978,0.024282,0.008192,0.109206,0.764485,9.586650e+07,"MULTIPOLYGON (((128.60966 35.15093, 128.60956 ..."
3,마산동부경찰서,창원,0.231992,0.131694,0.490015,0.803621,0.120852,0.096054,0.000656,0.120422,1.006756,9.217463e+07,"MULTIPOLYGON (((128.62696 35.21714, 128.62695 ..."
4,마산중부경찰서,창원,0.185716,0.116464,0.487745,0.834603,0.092932,0.144528,0.004446,0.098680,0.575626,2.342724e+08,"MULTIPOLYGON (((128.43007 35.05918, 128.43005 ..."
5,진해경찰서,창원,0.066315,0.015530,0.809750,0.886519,0.076231,0.021237,0.012894,0.039925,0.480173,1.214798e+08,"MULTIPOLYGON (((128.81143 35.07759, 128.80804 ..."
6,서울강동경찰서,서울,0.307764,0.503207,0.721685,0.675715,0.407134,0.183791,0.444362,0.440437,0.961316,2.340593e+07,"MULTIPOLYGON (((127.11397 37.55685, 127.11397 ..."
7,서울송파경찰서,서울,0.414543,0.499345,1.070241,0.900847,0.365791,0.107902,0.435112,0.199239,0.914660,3.074816e+07,"MULTIPOLYGON (((127.14542 37.51605, 127.14519 ..."
8,서울수서경찰서,서울,0.403432,0.340980,0.718830,0.999621,0.412666,0.059192,0.538111,0.559169,0.301423,2.515834e+07,"MULTIPOLYGON (((127.12420 37.46652, 127.12414 ..."
9,서울방배경찰서,서울,0.467214,0.322464,0.675568,0.849451,0.397149,0.219146,1.237705,1.418944,0.299471,9.229234e+06,"MULTIPOLYGON (((127.00365 37.46771, 127.00305 ..."


### 체감 안전도 feature
- 사회적 무질서: 질서유지 관련 112 데이터(112_질서유지)
- 물리적 무질서: 시설물 관련 데이터(시설물)
- 교통안전: 교통관련 데이터(112_교통)
- 경찰활동: 전반적경찰노력도_점수 데이터(전반적경찰노력도_점수)

### 각 feature들의 점수화를 이용하기 위하여 Natural Breaks 사용

In [45]:
for i in list(police_4sido.columns)[2:11]:
    breaks=jenkspy.jenks_breaks(police_4sido[i],nb_class=10)
    police_4sido[i]=pd.cut(police_4sido[i],bins=breaks,
                               labels=[i for i in range(1,11)],
                               include_lowest=True)
    police_4sido[i] =police_4sido[i].astype(int) 

### 경찰활동 관련 Feature 생성

In [46]:
safe1=pd.read_csv("체감안전도(20180101).csv", header=None)
safe2=pd.read_csv("체감안전도(20180701).csv", header=None)

In [47]:
safe_header=['대상년월',
'경찰서',
'설문대상인원_전체',
'설문대상인원_1급지',
'설문대상인원_2급지',
'설문대상인원_3급지',
'설문대상인원_남성',
'설문대상인원_여성',
'설문대상인원_20대이하',
'설문대상인원_30대',
'설문대상인원_40대',
'설문대상인원_50대',
'설문대상인원_60대이하',
'종합체감안전도_점수',
'전반적안전도_점수',
'분야별안전도_점수',
'범죄안전도_점수',
'교통사고안전도_점수',
'법질서준수도_점수',
'절도폭력안전도_점수',
'강도살인안전도_점수',
'전반적경찰노력도_점수']

In [48]:
safe=pd.concat([safe1, safe2], axis=0).reset_index(drop=True)
safe.columns=safe_header

In [49]:
safe=safe[['경찰서',
'종합체감안전도_점수',
'전반적안전도_점수',
'분야별안전도_점수',
'범죄안전도_점수',
'교통사고안전도_점수',
'법질서준수도_점수',
'절도폭력안전도_점수',
'강도살인안전도_점수',
'전반적경찰노력도_점수']]
safe_4 = safe[(safe["경찰서"].str[:2] == "서울") | 
            (safe["경찰서"].str[4:6] == "수원") | 
            (safe["경찰서"].str[2:4] == "창원") |
            (safe["경찰서"].str[2:4] == "마산") |
            (safe["경찰서"].str[2:4] == "진주")|
            (safe["경찰서"].str[2:4] == "진해") ]

In [50]:
safe_groupby=safe_4.groupby(['경찰서']).mean().reset_index()
safe_groupby.loc[0, '경찰서']="수원남부경찰서"
safe_groupby.loc[1, '경찰서']="수원서부경찰서"
safe_groupby.loc[2, '경찰서']="수원중부경찰서"
safe_groupby.loc[3, '경찰서']="마산동부경찰서"
safe_groupby.loc[4, '경찰서']="마산중부경찰서"
safe_groupby.loc[5, '경찰서']="진주경찰서"
safe_groupby.loc[6, '경찰서']="진해경찰서"
safe_groupby.loc[7, '경찰서']="창원서부경찰서"
safe_groupby.loc[8, '경찰서']="창원중부경찰서"
safe_groupby.sort_values(by="경찰서", inplace=True)
safe_groupby.reset_index(drop=True,inplace=True)
safe_groupby.rename(columns={'경찰서':'관할경찰서'},inplace = True)

In [51]:
#경찰활동 Feature 점수화
경찰활동 = safe_groupby[['관할경찰서','전반적경찰노력도_점수']]
경찰활동['전반적경찰노력도_점수'].describe()

breaks=jenkspy.jenks_breaks(경찰활동['전반적경찰노력도_점수'],nb_class=10)
경찰활동['전반적경찰노력도_점수']=pd.cut(경찰활동['전반적경찰노력도_점수'],bins=breaks,
                           labels=[i for i in range(1,11)],
                           include_lowest=True)
경찰활동['전반적경찰노력도_점수'] =경찰활동['전반적경찰노력도_점수'].astype(int) 

In [52]:
police_4sido = pd.merge(police_4sido,경찰활동,on = '관할경찰서')

물리적 무질서 
- 음의 시설물: '편의점','보안등','비상벨','cctv'
- 양의 시설물: '공원','놀이시설','유흥업소'
- 무질서가 높을수록 치안이 좋지 않다 

In [53]:
for i in ['편의점','보안등','비상벨','cctv','전반적경찰노력도_점수']:
    police_4sido[i]=police_4sido[i].replace([1,2,3,4,5,6,7,8,9,10],[10,9,8,7,6,5,4,3,2,1])

In [54]:
police_4sido['물리적_무질서'] = round(((police_4sido['편의점']+ police_4sido['보안등']+ police_4sido['비상벨']+ police_4sido['cctv']+ police_4sido['공원']+ police_4sido['놀이시설']+ police_4sido['유흥업소'] )/7),2 )
police_4sido['사회적_무질서'] = police_4sido['112_질서유지']
police_4sido['교통안전'] = police_4sido['112_교통']
police_4sido['경찰활동'] = police_4sido['전반적경찰노력도_점수']
police_4sido['체감안전도_지수'] = police_4sido['물리적_무질서']+police_4sido['사회적_무질서']+police_4sido['교통안전'] +police_4sido['경찰활동'] 

In [55]:
epsg4326 = from_string("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
police_4sido = gpd.GeoDataFrame(police_4sido, geometry='geometry', crs = epsg4326)
police_4sido.to_file('4개시도_체감안전도_지수.geojson',driver = 'GeoJSON')